In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
t_d=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
tt_d=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [5]:
batch_size=64
t_d_loader=DataLoader(t_d,batch_size=batch_size)
tt_d_loader=DataLoader(tt_d,batch_size=batch_size)

In [6]:
for X,y in tt_d_loader:
    print(f'Shape of X[n,c,h,w]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X[n,c,h,w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [21]:
def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    for batch,(X,y) in enumerate(dataloader):
        X,y=X.to(device),y.to(device)

        pred=model(X)
        loss=loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch %100==0:
            loss,current=loss.item(),batch*len(X)
            print(f'loss:{loss:>7f}[{current:>5d}/{size:>5d}]')

In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(t_d_loader, model, loss_fn, optimizer)
    test(tt_d_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.299652[    0/60000]
loss:2.290160[ 6400/60000]
loss:2.268256[12800/60000]
loss:2.265456[19200/60000]
loss:2.254279[25600/60000]
loss:2.205792[32000/60000]
loss:2.218231[38400/60000]
loss:2.179510[44800/60000]
loss:2.181664[51200/60000]
loss:2.140356[57600/60000]
Test Error: 
 Accuracy: 36.2%, Avg loss: 2.140348 

Epoch 2
-------------------------------
loss:2.150202[    0/60000]
loss:2.137284[ 6400/60000]
loss:2.079398[12800/60000]
loss:2.103313[19200/60000]
loss:2.045678[25600/60000]
loss:1.974494[32000/60000]
loss:2.002400[38400/60000]
loss:1.924976[44800/60000]
loss:1.941249[51200/60000]
loss:1.845607[57600/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 1.858335 

Epoch 3
-------------------------------
loss:1.895754[    0/60000]
loss:1.853969[ 6400/60000]
loss:1.748763[12800/60000]
loss:1.795357[19200/60000]
loss:1.677998[25600/60000]
loss:1.633170[32000/60000]
loss:1.647399[38400/60000]
loss:1.568750[44800/60000]
loss:1.596173[51200/